In [25]:
# To get multiple outputs from one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
from pathlib import Path

def sys_info():
    frmt = '\nPython ver: {}\nPython env: {}\n'
    frmt += 'OS:         {}\nCurrent dir: {}\n'
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = F'Path already in sys.path: {newp}'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = F'Path added to sys.path: {newp}'
    print(msg)

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


# Filtered dir() for method discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_project_dirs(which=['data', 'images'],
                     nb_folder='notebooks',
                     use_parent=True):
    '''Create folder(s) named in `which` at the parent level.'''
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder) or use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

DIR_DATA, DIR_IMG = get_project_dirs()
    
#import numpy as np
#import scipy as sp
#from scipy import stats as sps
import pandas as pd
#pd.set_option("display.max_colwidth", 200)

import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-muted')

from pprint import pprint as pp


def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)

    
def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')


def show_versions():
    txt = '<pre><br>'
    txt += F'Python:\t\t{sys.version}<br>'
    txt += F'Python env:\t{Path(sys.prefix).name}<br>'
    #txt += F'Numpy:\t\t{np.__version__}<br>'
    #txt += F'Scipy:\t\t{sp.__version__}<br>'
    #txt += F'Pandas:\t\t{pd.__version__}<br>'
    txt += F'Matplotlib:\t{mpl.__version__}<br>'
    txt += F'Currrent dir: {Path.cwd()}'
    txt += '</pre>'
    div = f"""<div class="alert alert-info"><b>Versions:</b><br>{txt}</div>"""
    return HTML(div)


# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2

#..................
sys_info()

no_wmark = False
try:
    %load_ext watermark
    %watermark
except ModuleNotFoundError:
    no_wmark = True

if no_wmark:
    show_versions()
else:
    %watermark -iv

Path added to sys.path: C:/Users/catch/Documents/GitHub/new_env

Python ver: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:33) [MSC v.1929 64 bit (AMD64)]
Python env: py310
OS:         win32
Current dir: C:\Users\catch\Documents\GitHub\new_env\notebooks



C:\Users\catch\AppData\Local\Temp\ipykernel_14396\775255977.py:87: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-muted')


---
---
# Testing locate.py (subprocess)
---

In [135]:
import os
from pathlib import Path
import subprocess
from subprocess import Popen, PIPE

import yaml
from yaml import Loader, Dumper


def get_yml_file(yml_path: Path) -> dict:
    if yml_path.exists():
        with open(yml_path) as stream:
            yml_file = yaml.load(stream, Loader)
        return yml_file
    raise FileNotFoundError(f"NotFound: {yml_path}")
    
    
def save_dict_to_yml(yml_filepath: Path, yml_dict: dict) -> None:
    with open(yml_filepath, 'w') as out:
        yaml.dump(yml_dict, out)

        
def get_python_dirs():
    """Run the command `where python` in the current directory.
    """
    with Popen(["where", "python"], stdout=PIPE, text=True) as proc:
        python_dirs = proc.stdout.read()
        
    return python_dirs


def get_user_dirs(dirs: list[str]):
    """Return a list of the directories' Path.home() part from a list of directories (str) 
    if their paths contain 'Users' or 'home'.
    """
    user_dirs = set()
    for d in dirs:
        if "Users" in d or "home" in d:
            user_dirs.add(Path(d).home())
    
    return list(user_dirs)


def get_new_prefix(u_dir:Path, new_env_name: str) -> str:
    """Return a yml prefix key value:
       prefix: u_dir\<Anaconda3|Miniconda3>\envs\new_env_name
    """
    # Anaconda3:
    conda_env_path = u_dir.joinpath("Anaconda3", "envs")
    if conda_env_path.exists():
        return str(conda_env_path.joinpath(new_env_name))
    # Miniconda3:
    conda_env_path = u_dir.joinpath("Miniconda3", "envs")
    if conda_env_path.exists():
        return str(conda_env_path.joinpath(new_env_name))
    # other cases: not yet supported, TODO
    msg = "Conda environments folder other than "
    msg += "'u_dir\Anaconda3\envs' or 'u_dir\Miniconda3\envs '"
    msg += "not yet supported."
    raise ValueError(msg)
    

def get_python_new_deps(env_file:dict, new_py_ver: str) -> list:
    """Add or amend the yaml dependencies for python.
    """
    
    deps = env_file["dependencies"]
    new_deps = []
    pys = [i for i, d in enumerate(deps) if d.startswith("python=")]
    if pys:
        del deps[pys[0]]  # in case there are several, possible??
   
    return ["python="+new_py_ver] + deps 


def create_new_env_yml(user_dir, new_py_ver, new_env_name):
    """
    """
    
def main() -> list:
    new_py_ver = "3.9"
    new_env_name = "fav39"
    
    # For testing, env_generic_minimal_test.yml has no python dep & prefix line is "prefix: envs_path\some_name"
    yml_filename = "env_generic_minimal_test.yml"
    yml_new_filename = yml_filename.replace("test", new_env_name)
    
    # todo: run conda export --from history > yml_filename if not .exists()

    python_dirs = get_python_dirs().split('\n')
    user_dirs = get_user_dirs(python_dirs)
    user_dir = user_dirs[0]                    # consider 1st one only
    yml_path = user_dir.joinpath(yml_filename) 
    
    # >>> in create_new_env_yml:
    new_prefix = get_new_prefix(user_dir, new_env_name)
    env_file = get_yml_file(yml_path)
    new_deps = get_python_new_deps(env_file, new_py_ver)

    # update dict:
    env_file["name"] = new_env_name
    env_file["dependencies"] = new_deps
    env_file["prefix"] = new_prefix
    # <<<<
    
    #todo: create new env file
    #create_new_env_yml(yml_path, env_file, new_py_ver, yml_new_filename, env_file)
    
    return
    

In [137]:
yml_path

WindowsPath('C:/Users/catch/env_generic_minimal_test.yml')

In [130]:
python_dirs = get_python_dirs().split('\n')
python_dirs

user_dirs = get_user_dirs(python_dirs)
user_dirs
str(user_dirs[0])

['C:\\Users\\catch\\Anaconda3\\envs\\py310\\python.exe', 'C:\\Program Files\\Python310\\python.exe', 'C:\\Users\\catch\\AppData\\Local\\Microsoft\\WindowsApps\\python.exe', '']

[WindowsPath('C:/Users/catch')]

'C:\\Users\\catch'

In [134]:
# For testing, env_generic_minimal_test.yml has no python dep & prefix line is "prefix: envs_path\some_name"
yml_filename = "env_generic_minimal_test.yml"
yml_path = user_dirs[0].joinpath(yml_filename)

env_file = get_yml_file(yml_path)
env_file.keys()

dict_keys(['name', 'channels', 'dependencies', 'prefix'])

In [127]:
pp(env_file)

{'channels': ['conda-forge', 'bioconda', 'pyviz', 'anaconda', 'defaults'],
 'dependencies': ['numpy',
                  'scipy',
                  'pandas',
                  'scikit-learn',
                  'ca-certificates',
                  'certifi',
                  'openssl',
                  'matplotlib',
                  'seaborn',
                  'panel',
                  'hvplot',
                  'nodejs',
                  'json',
                  'jupyterlab',
                  'traitlets',
                  'nbformat',
                  'jupyterlab_server',
                  'jupyter_server',
                  'click'],
 'name': 'some_name',
 'prefix': 'envs_path\\some_name'}


'env_generic_minimal_fav39.yml'